In [2]:
import gdp as getgdp
import pandas as pd

gdpdf = getgdp()


"""
Our analysis will focus on the long-term impact on the Canadian economy
regarding the cost of children/minors missing out on school, or getting bad starts on their school year.
This analysis is based on a study from the World Bank in May 2020 that focused on similar factors we'll
be highlighting today
Lost Wages - the COVID-19 Cost of School Closures
G. Psacharopoulos, V. Collis, H.A. Patrinos & E. Vegas
http://documents1.worldbank.org/curated/en/702401598463119393/pdf/Lost-Wages-The-COVID-19-Cost-of-School-Closures.pdfhttp://documents1.worldbank.org/curated/en/702401598463119393/pdf/Lost-Wages-The-COVID-19-Cost-of-School-Closures.pdf

This approach relies on Human Capital Theory, where it is stipulated that investments of time and money
on education have a Human Capital building effect similar if not identical to the effect of investing
in real-physical assets such as infrastructure.


To narrow down our sample of study to model the simulation, we'll be using survey data from Abacus that studied the 
month of September 2020 in Canada, to estimate how many students missed close to a month of school, or had
sub-par educational support from schools due to various pandemic factors, during a school years' most important 
month.
"""
cgdp = getgdp('All industries')
abc = pd.read_csv('data/abacus.csv')

# print(cgdp)
print(abc)



           Period      Value
287    1997-01-01  1161269.0
1148   1997-02-01  1168576.0
2009   1997-03-01  1168456.0
2870   1997-04-01  1176368.0
3731   1997-05-01  1180541.0
4592   1997-06-01  1182790.0
5453   1997-07-01  1195510.0
6314   1997-08-01  1197012.0
7175   1997-09-01  1199096.0
8036   1997-10-01  1203751.0
8897   1997-11-01  1204715.0
9758   1997-12-01  1215663.0
10619  1998-01-01  1210338.0
11480  1998-02-01  1222196.0
12341  1998-03-01  1226815.0
13202  1998-04-01  1226970.0
14063  1998-05-01  1224212.0
14924  1998-06-01  1223190.0
15785  1998-07-01  1222521.0
16646  1998-08-01  1234025.0
17507  1998-09-01  1237237.0
18368  1998-10-01  1241598.0
19229  1998-11-01  1244099.0
20090  1998-12-01  1254166.0
20951  1999-01-01  1264286.0
21812  1999-02-01  1269617.0
22673  1999-03-01  1270087.0
23534  1999-04-01  1276505.0
24395  1999-05-01  1278394.0
25256  1999-06-01  1286392.0
...           ...        ...
219842 2018-04-01  1943319.0
220703 2018-05-01  1953894.0
221564 2018-06

NameError: name 'abc' is not defined